### Imports

In [ ]:
from py2neo import Graph, Node
from tqdm.auto import tqdm
import datetime
import pandas as pd
import numpy as np 
import requests
import json
import praw
import psycopg2
from psycopg2.extras import Json
import datetime
import time

reddit_client_id = "_nGE8QZMxX__JA"
reddit_client_secret = "llH7txuFvKFHk4-LXb3XsM-wjbYNnA"

reddit = praw.Reddit(
    client_id = reddit_client_id,
    client_secret = reddit_client_secret,
    redirect = "http://localhost:8080",
    username = "LookAtAllThesePeeps",
    password = "Rogerthat_121091",
    user_agent = "personal use script"
)

print(reddit.user.me())

### Types

In [ ]:
if "submission_types" in locals():
    del submission_types
    
submission_types = {
 'approved_at_utc': 'timestamp with time zone',
 'approved_by': "text",
 'archived': "bool",
 'author': "text",
 'author_flair_background_color': 'text',
 'author_flair_css_class': 'text',
 'author_flair_richtext': 'text',
 'author_flair_template_id': 'int',
 'author_flair_text': 'text',
 'author_flair_text_color': 'text',
 'author_flair_type': 'text',
 'author_fullname': 'text',
 'author_patreon_flair': 'text',
 'author_premium': 'bool',
 'banned_at_utc': 'timestamp with time zone',
 'banned_by': 'text',
 'can_gild': 'bool',
 'can_mod_post': "bool",
 'category': "text",
 'clicked': 'bool',
 'comment_limit': "text",
 'comment_sort': "text",
 'content_categories': 'text',
 'contest_mode': 'text',
 'created': 'integer',
 'created_utc': 'timestamp without time zone',
 'discussion_type': "text",
 'distinguished': "bool",
 'domain': "text",
 'downs': "integer",
 'edited': 'bool',
 'fullname': "text",
 'gilded': "integer",
 'hidden': "bool",
 'id': "text",
 'is_crosspostable': "bool",
 'is_meta': "bool",
 'is_original_content': "bool",
 'is_reddit_media_domain': "bool",
 'is_robot_indexable': "bool",
 'is_self': "bool",
 'is_video': "bool",
 'likes': "integer",
 'link_flair_background_color': "text",
 'link_flair_css_class': "text",
 'link_flair_template_id': "text",
 'link_flair_text': 'text',
 'link_flair_text_color': 'text',
 'link_flair_type': 'text',
 'locked': "bool",
 'mod_note': "text",
 'mod_reason_by': "text",
 'mod_reason_title': 'text',
 'name': 'text',
 'no_follow': "bool",
 'num_comments': 'integer',
 'num_crossposts': 'integer',
 'num_reports': 'integer',
 'over_18': 'bool',
 'parent_whitelist_status': 'text',
 'permalink': 'text',
 'pinned': 'bool',
 'pwls': 'integer',
 'quarantine': 'bool',
 'removal_reason': 'text',
 'removed_by': 'text',
 'removed_by_category': 'text',
 'report_reasons': 'text',
 'saved': 'bool',
 'score': 'integer',
 'selftext': 'text',
 'selftext_html': 'text',
 'shortlink': 'text',
 'spoiler': 'bool',
 'stickied': "bool",
 'subreddit_id': "text",
 'subreddit_name_prefixed': 'text',
 'subreddit_subscribers': 'int',
 'subreddit_type': 'text',
 'suggested_sort': 'text',
 'thumbnail': 'text',
 'title': 'text',
 'top_awarded_type': 'text',
 'total_awards_received': 'integer',
 'ups': 'integer',
 'upvote_ratio': 'float',
 'url': 'text',
 'view_count': 'integer',
 'visited': 'bool',
 'whitelist_status': 'text',
 'wls': "integer"
}

comment_types = {
 'total_awards_received': 'int',
 'approved_at_utc': 'int',
 'comment_type': 'int',
 'awarders': 'jsonb',
 'mod_reason_by': 'text',
 'banned_by': 'text',
 'ups': "int",
 'author_flair_type': "text",
 'removal_reason': "text",
 'link_id': "text",
 'author_flair_template_id': "text",
 'likes': "int",
 'user_reports': "jsonb",
 'saved': "bool",
 'id': "text",
 'banned_at_utc': "int",
 'mod_reason_title': "text",
 'gilded': "bool",
 'archived': "bool",
 'no_follow': "bool",
 'author': "text",
 'can_mod_post': "bool",
 'send_replies': "bool",
 'parent_id': "text",
 'score': "int",
 'author_fullname': "text",
 'report_reasons': "jsonb",
 'approved_by': "text",
 'all_awardings': "jsonb",
 'subreddit_id': "text",
 'body': "text",
 'edited': "bool",
 'downs': "int",
 'author_flair_css_class': "text",
 'is_submitter': "bool",
 'collapsed': "bool",
 'author_flair_richtext': "jsonb",
 'author_patreon_flair': "text",
 'body_html': "text",
 'gildings': "jsonb",
 'collapsed_reason': "text",
 'associated_award': "text",
 'stickied': "bool",
 'author_premium': "bool",
 'subreddit_type': "text",
 'can_gild': "bool",
 'top_awarded_type': "text",
 'author_flair_text_color': "text",
 'score_hidden': "bool",
 'permalink': "text",
 'num_reports': "int",
 'locked': "bool",
 'name': "text",
 'created': "int",
 'author_flair_text': "text",
 'treatment_tags': "jsonb",
 'created_utc': "int",
 'subreddit_name_prefixed': "text",
 'controversiality': "int",
 'depth': "int",
 'author_flair_background_color': "text",
 'collapsed_because_crowd_control': "bool",
 'mod_reports': "jsonb",
 'mod_note': "text",
 'distinguished': "bool",
}

CREATE_WSB_COMMENTS_TABLE_QUERY = \
"""create table wsb_comments
(
    total_awards_received           int,
    approved_at_utc                 int,
    comment_type                    int,
    awarders                        jsonb,
    mod_reason_by                   text,
    banned_by                       text,
    ups                             int,
    author_flair_type               text,
    removal_reason                  text,
    link_id                         text,
    author_flair_template_id        text,
    likes                           int,
    user_reports                    jsonb,
    saved                           bool,
    id                              text,
    banned_at_utc                   int,
    mod_reason_title                text,
    gilded                          int,
    archived                        bool,
    no_follow                       bool,
    author                          text,
    can_mod_post                    bool,
    send_replies                    bool,
    parent_id                       text,
    score                           int,
    author_fullname                 text,
    report_reasons                  jsonb,
    approved_by                     text,
    all_awardings                   jsonb,
    subreddit_id                    text,
    body                            text,
    edited                          int,
    downs                           int,
    author_flair_css_class          text,
    is_submitter                    bool,
    collapsed                       bool,
    author_flair_richtext           jsonb,
    author_patreon_flair            text,
    body_html                       text,
    gildings                        jsonb,
    collapsed_reason                text,
    associated_award                text,
    stickied                        bool,
    author_premium                  bool,
    subreddit_type                  text,
    can_gild                        bool,
    top_awarded_type                text,
    author_flair_text_color         text,
    score_hidden                    bool,
    permalink                       text,
    num_reports                     int,
    locked                          bool,
    name                            text,
    created                         int,
    author_flair_text               text,
    treatment_tags                  jsonb,
    created_utc                     int,
    subreddit_name_prefixed         text,
    controversiality                int,
    depth                           int,
    author_flair_background_color   text,
    collapsed_because_crowd_control bool,
    mod_reports                     jsonb,
    mod_note                        text,
    distinguished                   text
);"""

### Functions 

In [ ]:
# Conn 
if "get_conn" in locals():
    del get_conn
    
def get_conn() -> psycopg2.connect:
    return psycopg2.connect(
        dbname="polygonio", 
        port="6432", 
        host="localhost", 
        password='rogerthat', 
        user="postgres"
    )

if "insert_submission" in locals():
    del insert_submission
    
def insert_submission(submission) -> None:
    cols = ('%s,' * (len(submission.keys()))).rstrip(",")
    insert_query = f"""INSERT INTO wsb_comments({','.join(submission.keys())}) VALUES %s ON CONFLICT (id) DO NOTHING;"""
    
    with get_conn() as conn:
        with conn.cursor() as cur:
            insert_query_2 = cur.mogrify(insert_query, tuple(submission.values()))
            print(insert_query_2)
            cur.execute(insert_query_2)

### Grab WSB

- Questions: 
    - What's the most mentioned ticker today? (Define what's a ticker, and groupby date)
    - What's the sentiment on the sentences that contain the tickers
    - Historically, which stocks have been mentioned? (What are the links to these submissions?) 
      - How much time before a stock is mentioned on WSB and then takes off ?

#### Create Table statement

In [ ]:
statement = 'create table wsb_comments ('

for key, val in comment_types.items():
    statement +=  f'\n{key} {val},'
    
statement = statement[:-1] + ');'
# create_index_statment = "CREATE UNIQUE INDEX "
print(statement)

#### Ideas for neo4j nodes

In [ ]:
potential_node_types = [
    "Submission"
    "Comment",
    "Account"
]

potential_properties = [
    "approved_at_utc", 
    "approved_by", 
    "author", 
    "created_utc", 
    "fullname",
    "id",
    "likes",
    "name",
    "quarantine",
    "subreddit_subscribers",
    "url",
    "view_count",
    "title",
    "downs",
    "ups",
    "likes",
    "subreddit_subscribers",
    "subreddit_type",
    "subreddit_name_prefixed"
]

potential_labels = [
    "edited", 
    "category",
    "created", 
    "crosspost", 
    "can_mod_post", 
    "quarantine", 
    "subreddit", 
    "top_awarded_type",
    "over_18",
    "mod",
    "is_original_content",
]

potential_relationships = [
    "IS_CHILD",
]

### Logic

In [ ]:
subreddit = reddit.subreddit("wallstreetbets")

june_2021_titles = {}
for submission in subreddit.search(f"and title:'Daily Discussion Thread for June 2021'"):
    june_2021_titles[submission.id] = submission.title
    
june_2021_titles = {k:v for k,v in june_2021_titles.items() if "Agricultural" not in v}

june_2021_titles["nqi9f6"] = "Daily Discussion Thread for June 02, 2021"
june_2021_titles["nr9r9t"] = "Daily Discussion Thread for June 03, 2021"
june_2021_titles

In [ ]:
other_titles = {}
for submission in subreddit.search(f"and title:'Daily Discussion Thread'"):
    other_titles[submission.id] = submission.title
    
other_titles = {k:v for k,v in other_titles.items() if "Agricultural" not in v}
other_titles

In [ ]:
if "clean_comment" in locals():
    del clean_comment
    
def clean_comment(comment):
    return {key: val for key, val in vars(comment).items() if key not in ["_replies", "_submission", "_reddit", "subreddit", "author", "_fetched"]}

if "traverse_comments" in locals():
    del traverse_comments
    
def traverse_comments(comment):
    if hasattr(comment, "comments"):
        for cc in tqdm(comment.comments(), leave=False):
            traverse_comments(comment=cc)
    else:
        cleaned_comment = clean_comment(comment=comment)
        for key, val in cleaned_comment.items():
            if key in ["awarders", "user_reports", "all_awardings", "report_reasons", "gildings", "author_flair_richtext", "treatment_tags", "mod_reports"]:
                cleaned_comment[key] = Json(json.dumps(val))
            if isinstance(cleaned_comment["edited"], bool):
                cleaned_comment["edited"] = 0
                
        insert_query = f"""INSERT INTO wsb_comments ({','.join(cleaned_comment.keys())}) VALUES ({ ('%s,' * len(cleaned_comment)).rstrip(',') }) ON CONFLICT (id) DO NOTHING;"""
        with get_conn() as conn:
            with conn.cursor() as cur:
                try:
                    mogrified_query = cur.mogrify(insert_query, tuple(cleaned_comment.values()))
                    cur.execute(mogrified_query)
                except psycopg2.errors.ProgrammingError as e:
                    print(cleaned_comment['id'])
                    print(e)

In [ ]:
comments = []
for sub_id, sub_title in tqdm(june_2021_titles.items()):
#     submission = reddit.submission(id="nr9r9t")
    for comment in tqdm(submission.comments):
        traverse_comments(comment=comment)